### Useful links
#### Folium:
#### https://python-visualization.github.io/folium/quickstart.html
#### https://python-visualization.github.io/folium/modules.html

#### OSM:
#### http://overpass-turbo.eu/
#### https://wiki.openstreetmap.org/wiki/Overpass_API/Overpass_QL#By_area_.28area.29
#### https://wiki.openstreetmap.org/wiki/Overpass_API/Language_Guide#Relations_2
#### https://github.com/tyrasd/osmtogeojson - OSM to GeoJson

#### Geopandas:
#### https://nbviewer.jupyter.org/github/geohackweek/tutorial_contents/blob/master/vector/notebooks/geopandas_intro.ipynb
#### https://geopandas.readthedocs.io/en/latest/gallery/index.html

#### Use cases:
#### http://andrewgaidus.com/leaflet_webmaps_python/ - Creating Web Maps in Python with GeoPandas and Folium (SF crime)
#### https://medium.com/starschema-blog/draw-a-map-of-the-districts-of-budapest-using-the-overpass-api-of-openstreetmap-and-python-bd0417469935 - Budapest districts map

### Get necessary dependencies

In [164]:
import geopandas as gpd
from geopandas import GeoDataFrame

import folium

import requests
import json
import pandas as pd

### Create GeoPandasDataFrame for choropleth GeoData 

In [165]:
# Let's read the GeoJSON file downloaded from OSM.ORG to a geopandas dataframe. This includes all neighborhoods of Budapest as multipolygons 
hoods=gpd.read_file(r"Budapest_Neighborhoods.geojson",encoding = "ISO-8859-1")
hoods

,id,@id,admin_level,boundary,name,name:de,type,wikidata,wikipedia,fixme,castle_type,heritage,heritage:operator,historic,name:hu,ref:whc,note,name:cs,name:en,name:es,name:it,name:nl,name:ro,old_name,highway,...,swing_gate:type,bollard,sloped_curb,length,kerb:height,level,opening_hours,tourism,source:date,traffic_sign:direction,stop,memorial,camera:direction,camera:mount,camera:type,man_made,surveillance,surveillance:type,surveillance:zone,level_crossing,material,colour,railway:milestone:catenary_mast,railway:position,geometry
0,relation/1746619,relation/1746619,10,administrative,Újlipótváros,Neu Leopoldistadt,boundary,Q570142,hu:Újlipótváros,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"POLYGON ((19.06313 47.52420, 19.06329 47.52456..."
1,relation/2700740,relation/2700740,10,administrative,Kelenvölgy,Scharfeseck,boundary,Q1106230,hu:Kelenvölgy,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"POLYGON ((19.02056 47.45826, 19.02046 47.45724..."
2,relation/2700782,relation/2700782,10,administrative,Albertfalva,Albertdorf,boundary,Q578669,hu:Albertfalva,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"POLYGON ((19.05011 47.44992, 19.04990 47.44997..."
3,relation/2700869,relation/2700869,10,administrative,Kelenföld,Krenfeld,boundary,Q770025,hu:Kelenföld,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"POLYGON ((19.06393 47.46193, 19.06692 47.46643..."
4,relation/2700985,relation/2700985,10,administrative,Nádorkert,None,boundary,Q1220173,hu:Nádorkert,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"POLYGON ((19.05605 47.46903, 19.05329 47.46954..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4881,node/6854960423,node/6854960423,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,crossing,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,POINT (19.12452 47.44478)
4882,node/6868285700,node/6868285700,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,POINT (19.02271 47.44523)
4883,node/6868285707,node/6868285707,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,POINT (19.01777 47.44776)
4884,node/6871827844,node/6871827844,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,crossing,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,POINT (19.18858 47.53184)


In [166]:
#Budapest has 203 neighborhoods, the imported file includes much more data.
#Let's boil down the dataframe to include only the neighborhoods' rows
hoods=hoods[hoods['type'] == "boundary"]
hoods

,id,@id,admin_level,boundary,name,name:de,type,wikidata,wikipedia,fixme,castle_type,heritage,heritage:operator,historic,name:hu,ref:whc,note,name:cs,name:en,name:es,name:it,name:nl,name:ro,old_name,highway,...,swing_gate:type,bollard,sloped_curb,length,kerb:height,level,opening_hours,tourism,source:date,traffic_sign:direction,stop,memorial,camera:direction,camera:mount,camera:type,man_made,surveillance,surveillance:type,surveillance:zone,level_crossing,material,colour,railway:milestone:catenary_mast,railway:position,geometry
0,relation/1746619,relation/1746619,10,administrative,Újlipótváros,Neu Leopoldistadt,boundary,Q570142,hu:Újlipótváros,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"POLYGON ((19.06313 47.52420, 19.06329 47.52456..."
1,relation/2700740,relation/2700740,10,administrative,Kelenvölgy,Scharfeseck,boundary,Q1106230,hu:Kelenvölgy,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"POLYGON ((19.02056 47.45826, 19.02046 47.45724..."
2,relation/2700782,relation/2700782,10,administrative,Albertfalva,Albertdorf,boundary,Q578669,hu:Albertfalva,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"POLYGON ((19.05011 47.44992, 19.04990 47.44997..."
3,relation/2700869,relation/2700869,10,administrative,Kelenföld,Krenfeld,boundary,Q770025,hu:Kelenföld,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"POLYGON ((19.06393 47.46193, 19.06692 47.46643..."
4,relation/2700985,relation/2700985,10,administrative,Nádorkert,None,boundary,Q1220173,hu:Nádorkert,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"POLYGON ((19.05605 47.46903, 19.05329 47.46954..."
5,relation/2702614,relation/2702614,10,administrative,Infopark,None,boundary,Q1097054,hu:Infopark,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"POLYGON ((19.05900 47.46869, 19.05976 47.46866..."
6,relation/2702615,relation/2702615,10,administrative,Lágymányos,Leutmannosch,boundary,Q1160072,hu:Lágymányos,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"POLYGON ((19.06734 47.46853, 19.06751 47.46977..."
7,relation/2702616,relation/2702616,10,administrative,Szentimreváros,None,boundary,Q780503,hu:Szentimreváros,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"POLYGON ((19.02998 47.48307, 19.02991 47.48300..."
8,relation/2702687,relation/2702687,10,administrative,Gellérthegy,Gerhardsberg,boundary,Q386977,hu:Gellérthegy,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"POLYGON ((19.03363 47.48879, 19.03341 47.48874..."
9,relation/2713748,relation/2713748,10,administrative,Gazdagrét,Reicheried,boundary,Q923038,hu:Gazdagrét,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None

In [167]:
#From the 259 columns we only need neighborhood names and their multipolygon's coordinates in 'geometry' column
hoods=hoods[['name','geometry']]
hoods

,name,geometry
0,Újlipótváros,"POLYGON ((19.06313 47.52420, 19.06329 47.52456..."
1,Kelenvölgy,"POLYGON ((19.02056 47.45826, 19.02046 47.45724..."
2,Albertfalva,"POLYGON ((19.05011 47.44992, 19.04990 47.44997..."
3,Kelenföld,"POLYGON ((19.06393 47.46193, 19.06692 47.46643..."
4,Nádorkert,"POLYGON ((19.05605 47.46903, 19.05329 47.46954..."
5,Infopark,"POLYGON ((19.05900 47.46869, 19.05976 47.46866..."
6,Lágymányos,"POLYGON ((19.06734 47.46853, 19.06751 47.46977..."
7,Szentimreváros,"POLYGON ((19.02998 47.48307, 19.02991 47.48300..."
8,Gellérthegy,"POLYGON ((19.03363 47.48879, 19.03341 47.48874..."
9,Gazdagrét,"POLYGON ((18.99171 47.47342, 18.99174 47.47336..."


In [168]:
#Let's create the GeoDataFrame, we need to use the right reference system for the map, espg:4326
hoods=gpd.GeoDataFrame(hoods,geometry='geometry')
hoods.crs = {'init' :'epsg:4326'}
hoods

,name,geometry
0,Újlipótváros,"POLYGON ((19.06313 47.52420, 19.06329 47.52456..."
1,Kelenvölgy,"POLYGON ((19.02056 47.45826, 19.02046 47.45724..."
2,Albertfalva,"POLYGON ((19.05011 47.44992, 19.04990 47.44997..."
3,Kelenföld,"POLYGON ((19.06393 47.46193, 19.06692 47.46643..."
4,Nádorkert,"POLYGON ((19.05605 47.46903, 19.05329 47.46954..."
5,Infopark,"POLYGON ((19.05900 47.46869, 19.05976 47.46866..."
6,Lágymányos,"POLYGON ((19.06734 47.46853, 19.06751 47.46977..."
7,Szentimreváros,"POLYGON ((19.02998 47.48307, 19.02991 47.48300..."
8,Gellérthegy,"POLYGON ((19.03363 47.48879, 19.03341 47.48874..."
9,Gazdagrét,"POLYGON ((18.99171 47.47342, 18.99174 47.47336..."


### Get data for number of cafes by neighborhood 

In [170]:
#call overpass API and get number of cafes per neighborhood
overpass_query = """
    [out:json];
    area[name="Budapest"][boundary];
    rel(area)[boundary][admin_level=10];
    map_to_area;
    foreach->.d(
       (.d;);out; 
       (node(area.d)[amenity=cafe];
        way(area.d)[amenity=cafe];
        relation(area.d)[amenity=cafe];);
       out count;
    );
    """
overpass_url = "http://overpass-api.de/api/interpreter"
response = requests.get(overpass_url,params={'data': overpass_query})

In [172]:
#Let's see what we got
cafes=response.json()
cafes

{'version': 0.6,
 'generator': 'Overpass API 0.7.55.7 8b86ff77',
 'osm3s': {'timestamp_osm_base': '2019-10-30T21:47:02Z',
  'timestamp_areas_base': '2019-10-30T20:58:02Z',
  'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'},
 'elements': [{'type': 'area',
   'id': 3603186063,
   'tags': {'admin_level': '10',
    'boundary': 'administrative',
    'fixme': '"XI. utca és meghosszabbított vonala" nem teljesen értelmezhető, a határvonal azon a szakaszon csak tipp',
    'name': 'Baross Gábor-telep',
    'type': 'boundary',
    'wikidata': 'Q739958',
    'wikipedia': 'hu:Baross Gábor-telep'}},
  {'type': 'count',
   'id': 0,
   'tags': {'nodes': '0',
    'ways': '0',
    'relations': '0',
    'areas': '0',
    'total': '0'}},
  {'type': 'area',
   'id': 3603186116,
   'tags': {'admin_level': '10',
    'boundary': 'administrative',
    'name': 'Budatétény',
    'name:de': 'Kleinteting',
    'type': 'boundary',
    'wikidata

In [173]:
#The essence is in elements
cafes=cafes['elements']
cafes

[{'type': 'area',
  'id': 3603186063,
  'tags': {'admin_level': '10',
   'boundary': 'administrative',
   'fixme': '"XI. utca és meghosszabbított vonala" nem teljesen értelmezhető, a határvonal azon a szakaszon csak tipp',
   'name': 'Baross Gábor-telep',
   'type': 'boundary',
   'wikidata': 'Q739958',
   'wikipedia': 'hu:Baross Gábor-telep'}},
 {'type': 'count',
  'id': 0,
  'tags': {'nodes': '0',
   'ways': '0',
   'relations': '0',
   'areas': '0',
   'total': '0'}},
 {'type': 'area',
  'id': 3603186116,
  'tags': {'admin_level': '10',
   'boundary': 'administrative',
   'name': 'Budatétény',
   'name:de': 'Kleinteting',
   'type': 'boundary',
   'wikidata': 'Q249072',
   'wikipedia': 'hu:Budatétény'}},
 {'type': 'count',
  'id': 0,
  'tags': {'nodes': '2',
   'ways': '0',
   'relations': '0',
   'areas': '0',
   'total': '2'}},
 {'type': 'area',
  'id': 3603186370,
  'tags': {'admin_level': '10',
   'boundary': 'administrative',
   'name': 'Budafok',
   'name:de': 'Promontor',
   

In [174]:
#Let's try to read this data to give it some structure
from pandas.io.json import json_normalize
cafes=json_normalize(cafes)
cafes

,type,id,tags.admin_level,tags.boundary,tags.fixme,tags.name,tags.type,tags.wikidata,tags.wikipedia,tags.nodes,tags.ways,tags.relations,tags.areas,tags.total,tags.name:de,tags.old_name,tags.name:hu,tags.name:cs,tags.name:en,tags.name:es,tags.name:it,tags.name:nl,tags.name:ro,tags.castle_type,tags.heritage,tags.heritage:operator,tags.historic,tags.ref:whc,tags.note
0,area,3603186063,10,administrative,"""XI. utca és meghosszabbított vonala"" nem telj...",Baross Gábor-telep,boundary,Q739958,hu:Baross Gábor-telep,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,count,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,area,3603186116,10,administrative,NaN,Budatétény,boundary,Q249072,hu:Budatétény,NaN,NaN,NaN,NaN,NaN,Kleinteting,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,count,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0,0,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,area,3603186370,10,administrative,NaN,Budafok,boundary,Q525100,hu:Budafok,NaN,NaN,NaN,NaN,NaN,Promontor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,count,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
402,area,3603237893,10,administrative,NaN,Pestújhely,boundary,Q1233483,hu:Pestújhely,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
403,count,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
404,area,3606485975,10,administrative,NaN,Székesdűlő,boundary,Q1302433,hu:Székesdűlő,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [175]:
#Now read it into a pandas dataframe
cafes=pd.DataFrame(cafes)
cafes

,type,id,tags.admin_level,tags.boundary,tags.fixme,tags.name,tags.type,tags.wikidata,tags.wikipedia,tags.nodes,tags.ways,tags.relations,tags.areas,tags.total,tags.name:de,tags.old_name,tags.name:hu,tags.name:cs,tags.name:en,tags.name:es,tags.name:it,tags.name:nl,tags.name:ro,tags.castle_type,tags.heritage,tags.heritage:operator,tags.historic,tags.ref:whc,tags.note
0,area,3603186063,10,administrative,"""XI. utca és meghosszabbított vonala"" nem telj...",Baross Gábor-telep,boundary,Q739958,hu:Baross Gábor-telep,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,count,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,area,3603186116,10,administrative,NaN,Budatétény,boundary,Q249072,hu:Budatétény,NaN,NaN,NaN,NaN,NaN,Kleinteting,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,count,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0,0,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,area,3603186370,10,administrative,NaN,Budafok,boundary,Q525100,hu:Budafok,NaN,NaN,NaN,NaN,NaN,Promontor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,count,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
402,area,3603237893,10,administrative,NaN,Pestújhely,boundary,Q1233483,hu:Pestújhely,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
403,count,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
404,area,3606485975,10,administrative,NaN,Székesdűlő,boundary,Q1302433,hu:Székesdűlő,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [176]:
#I want to look at all the columns so I can pick the important ones
pd.set_option('display.max_columns', 50)
cafes

,type,id,tags.admin_level,tags.boundary,tags.fixme,tags.name,tags.type,tags.wikidata,tags.wikipedia,tags.nodes,tags.ways,tags.relations,tags.areas,tags.total,tags.name:de,tags.old_name,tags.name:hu,tags.name:cs,tags.name:en,tags.name:es,tags.name:it,tags.name:nl,tags.name:ro,tags.castle_type,tags.heritage,tags.heritage:operator,tags.historic,tags.ref:whc,tags.note
0,area,3603186063,10,administrative,"""XI. utca és meghosszabbított vonala"" nem telj...",Baross Gábor-telep,boundary,Q739958,hu:Baross Gábor-telep,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,count,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,area,3603186116,10,administrative,NaN,Budatétény,boundary,Q249072,hu:Budatétény,NaN,NaN,NaN,NaN,NaN,Kleinteting,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,count,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0,0,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,area,3603186370,10,administrative,NaN,Budafok,boundary,Q525100,hu:Budafok,NaN,NaN,NaN,NaN,NaN,Promontor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,count,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
402,area,3603237893,10,administrative,NaN,Pestújhely,boundary,Q1233483,hu:Pestújhely,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
403,count,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
404,area,3606485975,10,administrative,NaN,Székesdűlő,boundary,Q1302433,hu:Székesdűlő,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [177]:
#Three columns will be enough, cafes count is below each neighborhood line
cafes=cafes[['type','tags.name','tags.total']]
cafes

,type,tags.name,tags.total
0,area,Baross Gábor-telep,NaN
1,count,NaN,0
2,area,Budatétény,NaN
3,count,NaN,2
4,area,Budafok,NaN
...,...,...,...
401,count,NaN,1
402,area,Pestújhely,NaN
403,count,NaN,1
404,area,Székesdűlő,NaN


In [178]:
#Paste all "count" one row above
for index_label, row_series in cafes.iterrows():
    if pd.isnull(cafes.at[index_label , 'tags.total']):
       cafes.at[index_label , 'tags.total']=cafes.at[index_label+1 , 'tags.total']
cafes

,type,tags.name,tags.total
0,area,Baross Gábor-telep,0
1,count,NaN,0
2,area,Budatétény,2
3,count,NaN,2
4,area,Budafok,1
...,...,...,...
401,count,NaN,1
402,area,Pestújhely,1
403,count,NaN,1
404,area,Székesdűlő,0


In [179]:
#We don't need count rows anymore
cafes=cafes.loc[cafes['type']=='area']
cafes

,type,tags.name,tags.total
0,area,Baross Gábor-telep,0
2,area,Budatétény,2
4,area,Budafok,1
6,area,Nagytétény,0
8,area,Csepel-Csillagtelep,2
10,area,Csepel-Királyerdő,3
12,area,Csepel-Rózsadomb,1
14,area,Csepel-Erdősor,1
16,area,Csepel-Háros,0
18,area,Soroksár-Újtelep,1


In [180]:
#Let's give more meaningful names for the columns
cafes = cafes.rename(columns = {"tags.name": "name", 
                                  "tags.total":"cafes"})
cafes

,type,name,cafes
0,area,Baross Gábor-telep,0
2,area,Budatétény,2
4,area,Budafok,1
6,area,Nagytétény,0
8,area,Csepel-Csillagtelep,2
10,area,Csepel-Királyerdő,3
12,area,Csepel-Rózsadomb,1
14,area,Csepel-Erdősor,1
16,area,Csepel-Háros,0
18,area,Soroksár-Újtelep,1


In [181]:
#2 columns will be sufficient, let's also reindex
cafes=cafes[['name','cafes']]
cafes=cafes.reset_index(drop=True)
cafes

,name,cafes
0,Baross Gábor-telep,0
1,Budatétény,2
2,Budafok,1
3,Nagytétény,0
4,Csepel-Csillagtelep,2
5,Csepel-Királyerdő,3
6,Csepel-Rózsadomb,1
7,Csepel-Erdősor,1
8,Csepel-Háros,0
9,Soroksár-Újtelep,1


In [182]:
#Want to check if all looks correct
pd.set_option('display.max_rows', 203)
cafes

,name,cafes
0,Baross Gábor-telep,0
1,Budatétény,2
2,Budafok,1
3,Nagytétény,0
4,Csepel-Csillagtelep,2
5,Csepel-Királyerdő,3
6,Csepel-Rózsadomb,1
7,Csepel-Erdősor,1
8,Csepel-Háros,0
9,Soroksár-Újtelep,1


In [187]:
#Let's check data type of cafes column, object data type wouldn't work with folium
print(cafes.cafes.values.dtype)

object


In [188]:
#As I thought, it is object, let's convert it to float
cafes["cafes"] = cafes.cafes.astype(float)
print(cafes.cafes.values.dtype)

float64


### Create Choropleth Map showing distribution of Cafes by Neighborhood in Budapest

In [225]:
budamap = folium.Map(location=[47.387913, 19.140236],tiles='Stamen Toner', zoom_start=10)
folium.Choropleth(
    geo_data=hoods,
    name='Cafes',
    data=cafes,
    columns=['name', 'cafes'],
    key_on='feature.properties.name',
    fill_color='OrRd',
    fill_opacity=0.9,
    line_opacity=0.3,
    legend_name='cafes'
).add_to(budamap)

folium.LayerControl().add_to(budamap)

budamap